In [1]:
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig

Selecting previously unselected package libgl1-mesa-glx:amd64.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../libgl1-mesa-glx_23.0.4-0ubuntu1~22.04.1_amd64.deb ...
Unpacking libgl1-mesa-glx:amd64 (23.0.4-0ubuntu1~22.04.1) ...
Selecting previously unselected package swig4.0.
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubuntu1_all.deb ...
Unpacking swig (4.0.2-1ubuntu1) ...
Setting up libgl1-mesa-glx:amd64 (23.0.4-0ubuntu1~22.04.1) ...
Setting up swig4.0 (4.0.2-1ubuntu1) ...
Setting up swig (4.0.2-1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
%%capture
!pip install -q condacolab wrds swig
!pip install lightning
!pip install cohere openai tiktoken
# import condacolab
# condacolab.install()
!pip install -U git+https://github.com/AI4Finance-Foundation/FinRL.git

In [5]:
import torch
import lightning as pl
import finrl
from finrl import config

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline



class FinRLLightning(pl.LightningModule):
    def __init__(self, start_date, end_date, ticker_list, total_timesteps):
        super(FinRLLightning, self).__init__()

        # Initialize and configure the custom environment
        self.df = YahooDownloader(start_date=start_date, end_date=end_date, ticker_list=ticker_list).fetch_data()
        self.fe = FeatureEngineer(
            use_technical_indicator=True,
            tech_indicator_list=config.INDICATORS,
            use_turbulence=False,
            user_defined_feature=False,
        )
        self.df = self.fe.preprocess_data(self.df)

        # Split data into training and evaluation sets
        self.trade = data_split(self.df, start_date, end_date)

        stock_dimension = len(self.trade.tic.unique())
        state_space = 1 + 2 * stock_dimension + len(config.INDICATORS) * stock_dimension
        self.env_kwargs = {
            "hmax": 100,
            "num_stock_shares": [0] * stock_dimension,
            "initial_amount": 1000000,
            "buy_cost_pct": 0.001,
            "sell_cost_pct": 0.001,
            "state_space": state_space,
            "stock_dim": stock_dimension,
            "tech_indicator_list": config.INDICATORS,
            "action_space": stock_dimension,
            "reward_scaling": 1e-4,
        }
        self.e_train_gym = StockTradingEnv(df=self.trade, **self.env_kwargs)

        # Initialize and configure the DRL agent
        self.agent = DRLAgent(env=self.e_train_gym)
        self.total_timesteps = total_timesteps

    # def train_dataloader(self):
    #     # Implement your data loading logic here.
    #     # Load and preprocess the training data.
    #     train_data = self.trade  # Assuming 'self.train' contains your training data
    #     dataset = Dataset(train_data)
    #     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    #     return dataloader

    def forward(self, x):
        # Define forward pass (if needed)
        pass

    def training_step(self, batch, batch_idx):
        # Define the training logic
        trained_model = self.agent.train("ppo", total_timesteps=self.total_timesteps)
        return None

    def configure_optimizers(self):
        # You don't need an optimizer for reinforcement learning typically
        return []

if __name__ == '__main__':
    # Set your parameters here
    start_date = "2000-01-01"
    end_date = "2022-01-01"
    ticker_list = ["AAPL", "MSFT"]
    total_timesteps = 20000

    # Create the Lightning model and trainer
    model = FinRLLightning(start_date, end_date, ticker_list, total_timesteps)
    trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=1)

    # Train the model
    # trainer.fit(model)

    # Run backtest and evaluate the model
    e_trade_gym = StockTradingEnv(df=model.trade, turbulence_threshold=250.0, **model.env_kwargs)
    df_account_value, df_actions = DRLAgent.DRL_prediction(model=model.agent, environment=e_trade_gym)
    # Continue with backtesting analysis...


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (11072, 8)


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Successfully added technical indicators


AttributeError: ignored